In [1]:
from datasets import load_dataset

dataset = load_dataset("microsoft/ms_marco","v1.1")

Generating validation split:   0%|          | 0/10047 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/82326 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9650 [00:00<?, ? examples/s]

In [2]:
len(dataset['train'])

82326

In [3]:
len(dataset['test'])

9650

In [9]:
from tqdm import tqdm 
total_passages = set()
for r in dataset['test']:
    for t in r['passages']['passage_text']:
        total_passages.add(t)
for r in dataset['train']:
    for t in r['passages']['passage_text']:
        total_passages.add(t)
for r in dataset['validation']:
    for t in r['passages']['passage_text']:
        total_passages.add(t)

In [10]:
print(len(total_passages))

767675


In [20]:
d = dataset['validation']
i = 200
negatives = [t for idx, t in enumerate(d[i]['passages']['passage_text']) if d[i]['passages']['is_selected'][idx]==0]
positives = [t for idx, t in enumerate(d[i]['passages']['passage_text']) if d[i]['passages']['is_selected'][idx]==1]
print("Question:",d[i]['query'])
print()
print("True Passage")
for p in positives:
    print(p)
print()
print("False Passage")
for idx, n in enumerate(negatives):
    print(n)

Question: who wrote Nothing compares to you

True Passage
Nothing Compares 2 U  is a song originally written and composed by Prince for one of his side projects, The Family. It was later made famous by Irish recording artist Sinead Sinéad'O, connor whose arrangement was released as the second single from her second studio, Album I Do Not Want What I'haven T. got Speaking about her relationship with Prince in an interview with Norwegian station NRK in November 2014 Sinead sinéad, Said i did meet him a couple of. Times we'didn t get on at. All in, fact we had a-punch.. Up she: Continued he summoned me to his house 'After Nothing compares 2.' U i made it without. him

False Passage
Answer by Mecoach50. Confidence votes 22. Nothing Compares 2 U was written by Prince and recorded by Sinead O'Connor. Yes it was written by Prince, but no it was not first recorded by Sinead O' Connor. The first to record the song was a Prince created band called The Family. The song It Had To Be You was origin

In [2]:
from datasets import load_dataset
qap = load_dataset('enelpol/rag-mini-bioasq', 'question-answer-passages')
corpus = load_dataset('enelpol/rag-mini-bioasq', 'text-corpus')

Generating train split:   0%|          | 0/4012 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/707 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/40181 [00:00<?, ? examples/s]

In [9]:
from datasets import concatenate_datasets
q2 = qap['train'].rename_column('relevant_passage_ids','gold_titles')
q3 = qap['test'].rename_column('relevant_passage_ids','gold_titles')
q= concatenate_datasets([q2,q3])
q.to_json('data/mini-bioasq_query.jsonl',orient='records',lines=True)

Creating json from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

2102860

In [7]:
c2 = corpus['test'].rename_column('id','title')
c2.to_json('data/mini-bioasq_t2p.jsonl', orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/41 [00:00<?, ?ba/s]

61976973

In [1]:

import transformers

from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
import torch
import argparse
import re
from functools import partial
import logging
from datasets import load_dataset, Dataset
from tqdm import tqdm
import os
from torch.utils.data import DataLoader
import json
import copy

import pandas as pd

os.environ['CUDA_VISIBLE_DEVICES']='2'
t2p='data/2wikimultihopqa_dev_t2p.jsonl'
vector='data/2wikimultihopqa.pt'
query='data/2wikimultihopqa_dev_query_1000.jsonl'
result='data/2wikimultihopqa_dev_query_1000_scored.jsonl'

def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings

def encode_batch(data, tokenizer, model, batch_size=128):
    embeddings = []
    with torch.no_grad():
        for i in tqdm(range(0, len(data), batch_size)):
            batch = data[i:i + batch_size]
            inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to('cuda')
            outputs = model(**inputs)
            batch_embeddings = mean_pooling(outputs.last_hidden_state, inputs['attention_mask'])
            embeddings.append(batch_embeddings.cpu())
    return torch.cat(embeddings, dim=0)

encoder_model = 'facebook/contriever'

def get_similarities(search_space, query, model, tokenizer):
    with torch.no_grad():
        query_inputs = tokenizer(query, return_tensors="pt", padding=True, truncation=True).to('cuda')
        query_embedding = mean_pooling(model(**query_inputs).last_hidden_state,query_inputs['attention_mask']).cpu()  # Move to CPU
        query_embedding = query_embedding / query_embedding.norm(dim=1, keepdim=True)
        search_space = search_space / search_space.norm(dim=1, keepdim=True)
        similarities = torch.matmul(search_space, query_embedding.T).squeeze()
        return similarities
    

def map_nearest(row,space,model,tokenizer):
    l = get_similarities(space,row['question'],model,tokenizer)
    row["score"] = l
    return row

tokenizer = AutoTokenizer.from_pretrained(encoder_model)
model = AutoModel.from_pretrained(encoder_model).to('cuda')

if os.path.exists(vector):
    vec_passages=torch.load(vector)
else:
    with open(t2p, 'r', encoding='utf-8') as file:
        data = [json.loads(line)['passage'] for line in file]
    vec_passages = encode_batch(data, tokenizer, model)
    torch.save(vec_passages, vector)

dataset = load_dataset('json', data_files=query)["train"]
dataset = dataset.map(partial(map_nearest,space=vec_passages,model=model,tokenizer=tokenizer,))
dataset.to_json(result, orient="records", lines=True)

/home/tkdrnjs0621/miniconda3/envs/torch_241/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/tmp/ipykernel_1546167/1954674706.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be al

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

torch.Size([54957])


AssertionError: 

In [ ]:

import transformers

from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
import torch
import argparse
import re
from functools import partial
import logging
from datasets import load_dataset, Dataset
from tqdm import tqdm
import os
from torch.utils.data import DataLoader
import json
import copy

import pandas as pd

dataset = load_dataset('json', data_files=query)["train"]

In [30]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
from transformers import LukeTokenizer, LukeModel, LukeForEntityPairClassification

model = LukeModel.from_pretrained("studio-ousia/luke-base")
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-base")

text = "Beyoncé lives in Los Angeles."

entities = [
    "Beyoncé",
    "Los Angeles",
]  # Wikipedia entity titles corresponding to the entity mentions "Beyoncé" and "Los Angeles"
entity_spans = [(0, 7), (17, 28)]  # character-based entity spans corresponding to "Beyoncé" and "Los Angeles"
inputs = tokenizer(text, entities=entities, entity_spans=entity_spans, add_prefix_space=True, return_tensors="pt")
print(inputs)
print(tokenizer.decode(token_ids=inputs['input_ids'][0]))
outputs = model(**inputs)
word_last_hidden_state = outputs.last_hidden_state
entity_last_hidden_state = outputs.entity_last_hidden_state

model = LukeForEntityPairClassification.from_pretrained("studio-ousia/luke-large-finetuned-tacred")
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-tacred")
entity_spans = [(0, 7), (17, 28)]  # character-based entity spans corresponding to "Beyoncé" and "Los Angeles"
inputs = tokenizer(text, entity_spans=entity_spans, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
predicted_class_idx = int(logits[0].argmax())
print("Predicted class:", model.config.id2label[predicted_class_idx])

{'input_ids': tensor([[    0, 12674, 12695,  1074,    11,  1287,  1422,   479,     2]]), 'entity_ids': tensor([[1657,   32]]), 'entity_position_ids': tensor([[[ 1,  2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
          -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
         [ 5,  6, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
          -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'entity_attention_mask': tensor([[1, 1]])}
<s> Beyoncé lives in Los Angeles.</s>


Some weights of the model checkpoint at studio-ousia/luke-large-finetuned-tacred were not used when initializing LukeForEntityPairClassification: ['luke.embeddings.position_ids']
- This IS expected if you are initializing LukeForEntityPairClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntityPairClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Predicted class: per:cities_of_residence


In [7]:
from transformers import AutoTokenizer, LukeForTokenClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("studio-ousia/luke-base")
model = LukeForTokenClassification.from_pretrained("studio-ousia/luke-base")

inputs = tokenizer(
    "HuggingFace is a company based in Paris and New York", add_special_tokens=False, return_tensors="pt"
)

with torch.no_grad():
    logits = model(**inputs).logits

predicted_token_class_ids = logits.argmax(-1)

# Note that tokens are classified rather then input words which means that
# there might be more predicted token classes than words.
# Multiple token classes might account for the same word
predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]

labels = predicted_token_class_ids
loss = model(**inputs, labels=labels).loss
print(labels)

/home/tkdrnjs0621/miniconda3/envs/torch_241/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of LukeForTokenClassification were not initialized from the model checkpoint at studio-ousia/luke-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])


In [18]:
from transformers import AutoTokenizer, LukeForTokenClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("studio-ousia/luke-base")
model = LukeForTokenClassification.from_pretrained("studio-ousia/luke-base")

# Input text with [MASK] token
text = "<s>The capital of France is Paris.</s> France <mask>"
# print(tokenizer.decode(tokenizer.separation_token_id))
# Tokenize input
print(tokenizer.all_special_tokens)
input_ids = tokenizer.encode(text, return_tensors="pt")
masked_index = (input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1].item()

# Get predictions
with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs.logits

# Decode predicted token
predicted_token_id = predictions[0, masked_index].argmax(dim=-1).item()
predicted_token = tokenizer.decode([predicted_token_id])

print(f"Predicted word: {predicted_token}")


/home/tkdrnjs0621/miniconda3/envs/torch_241/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of LukeForTokenClassification were not initialized from the model checkpoint at studio-ousia/luke-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['<s>', '</s>', '<unk>', '<pad>', '<mask>', '<ent>', '<ent2>', '<ent>', '<ent2>']
Predicted word: <pad>


In [5]:
from datasets import load_dataset
dataset_path = 'data/entityrag/2wikimultihopqa_result_or2f.jsonl'
dataset = load_dataset('json', data_files=dataset_path)["train"]
a = 0 
for row in dataset:
    for gt in row['gold_titles']:
        gnt = gt
        if('(' in gnt):
            gnt = gnt.split('(')[0]
        gnt = gnt.strip().lower()
        if gnt in row['question'].lower():
            a+=1
            break
print(a, len(dataset), a/len(dataset)*100)

1000 1000 100.0
